In [3]:
import os
import sys
import yaml
import json
import numpy as np

with open("config.yaml", 'r') as yml_file:
    cfg = yaml.safe_load(yml_file)
    
#Get data path from config 
data_path = cfg['brian_laptop']['data_path']
mb_div = cfg['contants']['mb_div']

#Get dyype for meta file 
meta_data_file = os.path.join(data_path, cfg['data']['metadata'])
with open(meta_data_file) as f:
    meta_data = json.load(f)

In [2]:
import pandas as pd
print(pd.__version__)
import datetime as dt

0.25.1


## Tiny Data
150 rows, 3 cols

In [12]:
%%time

iris_file = os.path.join(data_path, cfg['data']['iris_csv'])
iris = pd.read_csv(iris_file)

CPU times: user 2.88 ms, sys: 986 µs, total: 3.86 ms
Wall time: 3.05 ms


In [13]:
print("data frame memory(MB):", round(sys.getsizeof(iris))/mb_div)
print("data frame memory(MB):", round(iris.memory_usage(deep = "True").sum()/mb_div,5))
print(iris.shape)
del(iris)

data frame memory(MB): 0.01476
data frame memory(MB): 0.01473
(150, 5)


 ## Bigger Data
55 million rows, 8 cols

In [14]:
%%time

#Take out the parse dates as to0 slow. 
nyc_taxi_fare_file = os.path.join(data_path, cfg['data']['kaggle_nyc'])
nyc_taxi_fare_dtype = dict(zip(meta_data['kaggle_nyc']['colnames'], meta_data['kaggle_nyc']['dtype']))
nyc_taxi_fare = pd.read_csv(nyc_taxi_fare_file, 
                            dtype = nyc_taxi_fare_dtype)

CPU times: user 1min 35s, sys: 1min 11s, total: 2min 46s
Wall time: 2min 53s


In [15]:
%%time

print("data frame memory(MB):", round(sys.getsizeof(nyc_taxi_fare)/mb_div,3))
print(nyc_taxi_fare.shape)
nyc_taxi_fare.head()

data frame memory(MB): 11787.478
(55423856, 8)


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [16]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].agg(count = len, 
                                                              mean = np.mean,
                                                              std = np.std,
                                                              min = np.min,
                                                              median = np.median,
                                                              max = np.max)

#How to get quantile ? 

CPU times: user 4.18 s, sys: 4.23 s, total: 8.41 s
Wall time: 9.63 s


,count,mean,std,min,median,max
passenger_count,,,,,,
0,195416.0,9.015344,6.512542,-12.50,7.3,259.00
1,38337524.0,11.216405,18.584711,-300.00,8.5,75747.02
2,8175243.0,11.838421,34.443510,-176.00,8.5,93963.36
3,2432712.0,11.540597,9.816888,-300.00,8.5,500.00
4,1178852.0,11.766069,10.196928,-57.33,8.5,978.00
5,3929346.0,11.208401,9.342562,-67.50,8.5,1021.30
6,1174647.0,12.126156,10.110923,-60.00,9.0,477.00
7,15.0,31.788667,30.066021,2.50,14.5,88.00
8,9.0,29.981111,34.045330,2.50,8.0,85.00


In [17]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].describe()

CPU times: user 9.86 s, sys: 3.06 s, total: 12.9 s
Wall time: 6.23 s


,count,mean,std,min,25%,50%,75%,max
passenger_count,,,,,,,,
0,195416.0,9.015344,6.512542,-12.50,5.30,7.3,10.50,259.00
1,38337524.0,11.216405,18.584711,-300.00,6.00,8.5,12.50,75747.02
2,8175243.0,11.838421,34.443510,-176.00,6.10,8.5,13.00,93963.36
3,2432712.0,11.540597,9.816888,-300.00,6.10,8.5,12.90,500.00
4,1178852.0,11.766069,10.196928,-57.33,6.10,8.5,13.00,978.00
5,3929346.0,11.208401,9.342562,-67.50,6.00,8.5,12.50,1021.30
6,1174647.0,12.126156,10.110923,-60.00,6.50,9.0,13.50,477.00
7,15.0,31.788667,30.066021,2.50,9.25,14.5,56.50,88.00
8,9.0,29.981111,34.045330,2.50,8.00,8.0,57.33,85.00


In [18]:
del(nyc_taxi_fare)

## PANDAS INTERESTING BITS ##

In [ ]:
nyc_taxi_fare.head()

## Too Big Data

46 million rows, 32 columns 
Need tp come up with a better example


In [73]:
%%time

dc_taxi_file_quad = os.path.join(data_path, cfg['data']['dc_taxi_2017_quad'])
dc_taxi_quad = pd.read_csv(dc_taxi_file_quad)

CPU times: user 6min 45s, sys: 6min 44s, total: 13min 30s
Wall time: 21min 40s


In [77]:
print("pandas table memory(MB)", round(sys.getsizeof(dc_taxi_quad)/mb_div,3))
print(dc_taxi_quad.shape)

NameError: name 'sys' is not defined

In [75]:
del(dc_taxi_quad)